## Connect to your Google Drive




In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Import revelant packages

In [2]:
!pip install flair
!pip install stop_words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import nltk
from datetime import datetime
import re
from stop_words import get_stop_words
from nltk import word_tokenize
from datetime import datetime
from stop_words import get_stop_words
nltk.download('stopwords')
nltk.download('punkt')
from nltk import word_tokenize, sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Define cleaning functions

In [4]:
import nltk
nltk.download('punkt')        # already done — but let's repeat
nltk.download('punkt_tab')    # this is the one missing!

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
#Create a column for the dates  of each post
def create_date_columns(data, DATE_COLUMN):
    date_utc = pd.to_datetime(data[DATE_COLUMN], format='%Y-%m-%d',utc=True)
    data['Year'] = date_utc.dt.strftime('%Y')
    data['Month'] = date_utc.dt.strftime('%Y-%m')
    data['Week'] = date_utc.dt.to_period("W").dt.start_time
    data['Day'] = date_utc.dt.strftime('%Y-%m-%d')
    return data

#Removes any mentions of social media handles from the text column
def replace_mentions(text):
    if text:
        new_text=[]
        for token in text.split(" "):
            token = "" if token.startswith('@') and len(token)>1 else token
            new_text.append(token)
        return " ".join(new_text)
    else:
        return text

#removes stopwords like 'or' & 'and' from the text column
def remove_stopwords(text,LANGUAGE):
    stopwords = get_stop_words(LANGUAGE.lower())
    return " ".join([token for token in word_tokenize(text) if token.lower() not in stopwords])

#Removes any emails or phone number from the text column
def replace_email_phone_links(text):
    text = re.sub('([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,})', "<email>", text)
    text = re.sub('(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})',"<tel>",text)
    text = re.sub('(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])', "<link>", text)
    return text

def basic_preprocessing(texts,LANGUAGE,keyword="",del_stopwords=True):
    texts = texts.apply(replace_email_phone_links)
    # remove tel and email
    texts = texts.str.replace("<link>","")
    texts = texts.str.replace("<email>","")
    texts = texts.str.replace("<tel>","")
    texts = texts.str.replace(keyword,"")
    if del_stopwords:
        texts = [remove_stopwords(text,LANGUAGE) for text in texts]
    # Remove new line characters
    texts = [re.sub('\s+', ' ', t) for t in texts]
    # Remove single quotes
    texts = [re.sub("\'", "", sent) for sent in texts]
    #Make all text lowercase
    texts = [text.lower() for text in texts]
    return texts


## Define path and data columns

In [15]:
PATH= "/content/drive/MyDrive/sundarbans/" # add here the correct path on Google Drive

#Optional organisation:

DATE_COLUMN = "created_at"
TEXT_COLUMN = "q_39" # adapt to the correct column name for Facebook and Instagram
AUTHOR_COLUMN = "Account"
LIKES_COLUMN = "Likes"
REACTIONS_COLUMN = "Total.Interactions"
LANGUAGE = "english"
REGEX = u'[A-Za-zÀ-ú]+' # words only

## Load the dataset

In [16]:
df = pd.read_csv(PATH+"topic_modelling.csv") # add here the correct data file name/content/drive/MyDrive/FileName.csv

## Clean the data

In [17]:
#drop empty cells
df = df.dropna(axis=1,how="all")
df = df.drop_duplicates()
df[TEXT_COLUMN] = df[TEXT_COLUMN].fillna("")
df["Number of Posts"] = 1

In [ ]:
#Optional: Create separate columns for month, week, day

df = create_date_columns(df, DATE_COLUMN) # generates 3 columns: Month, Week, Day
df.columns

ValueError: unconverted data remains when parsing with format "%Y-%m-%d": " 21:17:42+00:00", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [10]:
#double check if it works
df.head()

,q_0,q_1,q_2,q_3,q_4,q_5,q_6,q_7,q_8,q_9_1,...,q_35_3,q_36,q_37_1,q_37_2,q_37_3,q_37_4,q_37_5,q_38,q_39,Number of Posts
0,1.0,Alok Mandal,Purbo Malekan Ghumti 24 North Parganas Pin: 74...,8967918674,37.0,2.0,3.0,Mason,4.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,"Education is affected, low voltage causes prob...",Incense stick factory for women's financial in...,1
1,1.0,Amit Mandal,Malekan Ghumti,8116674320,42.0,2.0,3.0,Farming,4.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,"Not getting sleep, education is affected","Good electricity supply, sewing machine factory",1
2,1.0,Anima Mandal,Malekan Ghumti,8927962837,50.0,1.0,3.0,Farming,4.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,"Low voltage causing problems in farming, educ...",Solar power facility next to the house,1
3,1.0,Anjana Raptan,Malekan Ghumti,7319067689,62.0,1.0,3.0,Making paper tongs,2.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,"Cooking in the evening is difficult, unable to...","Good doctor, water purifier is necessary",1
4,1.0,Archana Mandal,Malekan Ghumti,7602570857,80.0,1.0,2.0,Migrant worker,5.0,1.0,...,0.0,0.0,1.0,1.0,1.0,1.0,1.0,Sleeping at night is difficult due to low voltage,Suggests using alternate energy sources like s...,1


## Run the functions

In [18]:
df[TEXT_COLUMN] = basic_preprocessing(df[TEXT_COLUMN],LANGUAGE,keyword="",del_stopwords=True)

In [19]:
df[TEXT_COLUMN] = [replace_mentions(text) for text in df[TEXT_COLUMN]]

## Save the cleaned dataset to drive



In [20]:
df.to_csv(PATH+"tm_clean2.csv", sep=',', encoding='utf-8')#choose file name and save